<a href="https://colab.research.google.com/github/MathewNhari/Sendy-Logistics-Challenge/blob/master/Final_Data_Cleaning_Ensemble%20Method%20running.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Importing Libraries**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#**Importing Data**

In [2]:
riders = pd.read_csv('Riders.csv')
SampleSubmission = pd.read_csv('SampleSubmission.csv')
Test_df = pd.read_csv('Test.csv')
Train_df = pd.read_csv('Train.csv', parse_dates= ['Placement - Day of Month'])
VariableDefinitions = pd.read_csv('VariableDefinitions.csv')

In [3]:
VariableDefinitions

,Order No,Unique number identifying the order
0,User Id,Unique number identifying the customer on a pl...
1,Vehicle Type,"For this competition limited to bikes, however..."
2,Platform Type,"Platform used to place the order, there are 4 ..."
3,Personal or Business,Customer type
4,Placement - Day of Month,Placement - Day of Month i.e 1-31
5,Placement - Weekday (Mo = 1),Placement - Weekday (Monday = 1)
6,Placement - Time,Placement - Time - Time of day the order was p...
7,Confirmation - Day of Month,Confirmation - Day of Month i.e 1-31
8,Confirmation - Weekday (Mo = 1),Confirmation - Weekday (Monday = 1)
9,Confirmation - Time,Confirmation - Time - Time of day the order wa...


**Sample Submission**

In [4]:
SampleSubmission.head(1)

,Order_No,Time from Pickup to Arrival
0,Order_No_19248,567.0


In [5]:
riders.head(1)

,Rider Id,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Rider_Id_396,2946,2298,14.0,1159


In [6]:
Train_df.head(1)

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Arrival at Pickup - Time,Pickup - Day of Month,Pickup - Weekday (Mo = 1),Pickup - Time,Arrival at Destination - Day of Month,Arrival at Destination - Weekday (Mo = 1),Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,9:35:46 AM,9,5,9:40:10 AM,9,5,10:04:47 AM,9,5,10:27:30 AM,9,5,10:39:55 AM,4,20.4,NaN,-1.317755,36.83037,-1.300406,36.829741,Rider_Id_432,745


In [7]:
Train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21201 entries, 0 to 21200
Data columns (total 29 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Order No                                   21201 non-null  object 
 1   User Id                                    21201 non-null  object 
 2   Vehicle Type                               21201 non-null  object 
 3   Platform Type                              21201 non-null  int64  
 4   Personal or Business                       21201 non-null  object 
 5   Placement - Day of Month                   21201 non-null  object 
 6   Placement - Weekday (Mo = 1)               21201 non-null  int64  
 7   Placement - Time                           21201 non-null  object 
 8   Confirmation - Day of Month                21201 non-null  int64  
 9   Confirmation - Weekday (Mo = 1)            21201 non-null  int64  
 10  Confirmation - Time   

In [8]:
#Train_df = Train_df.drop(['Precipitation in millimeters'], axis= 1)
#Test_df = Test_df.drop(['Precipitation in millimeters'], axis= 1)

# Joining Tables

In [9]:
Train_df = Train_df.merge(riders, on='Rider Id', how='left')
Test_df = Test_df.merge(riders, on='Rider Id', how='left')

# **Data Cleaning**

**Checking for Duplicates**

In [10]:
#No duplicate Data On Test Data
def Check_duplicates(df):
  duplicates = df.duplicated(subset = ['Order No'], keep = False)
  return df[duplicates]

In [11]:
#No duplicates 
Train_duplicates = Check_duplicates(Train_df)
Test_duplucates = Check_duplicates(Test_df)

**Changing into int data types

In [12]:
Train_df['Platform Type'] = Train_df['Platform Type'].astype('object')
Test_df['Platform Type'] = Test_df['Platform Type'].astype('object')       
Train_df['Placement - Day of Month'] = Train_df['Placement - Day of Month'].astype('int')
Test_df['Placement - Day of Month'] = Test_df['Placement - Day of Month'].astype('int')

**Dropping columns not in test Data**

In [13]:
def columnDrop(df,columns):
  df.drop(columns, axis=1,inplace=True)
  #df.set_index(['Order No'], inplace= True)
  return df

In [14]:
Drop_Train_Cols = ['Precipitation in millimeters', 'Arrival at Destination - Day of Month', 
                   'Arrival at Destination - Weekday (Mo = 1)', 'Arrival at Destination - Time']
Drop_Test_Cols =  ['Precipitation in millimeters']    
Test_df = columnDrop(Test_df,Drop_Test_Cols)
Train_df = columnDrop(Train_df,Drop_Train_Cols)

**Seperating Time**

In [15]:
def timeconvert(df,Time):
    df[Time] = df[Time].astype('datetime64')
    return df

In [16]:
Time_columns = ['Placement - Time', 'Confirmation - Time',
                'Arrival at Pickup - Time','Pickup - Time']
Train_df = timeconvert(Train_df,Time_columns)
Test_df = timeconvert(Test_df,Time_columns)

In [17]:
def SplitDatetime(df,columns):
  for item in columns:
    #df[item+'_day'] = pd.to_datetime(df[item]).dt.day
    df[item+'_hour'] = pd.to_datetime(df[item]).dt.hour
    df[item+'_minutes'] = pd.to_datetime(df[item]).dt.minute
    df[item+'_seconds'] = pd.to_datetime(df[item]).dt.second
  df.drop(columns,axis=1, inplace = True)
  return df

In [18]:
Time_columns = ['Placement - Time', 'Confirmation - Time',
                'Arrival at Pickup - Time','Pickup - Time']
Train_df = SplitDatetime(Train_df,Time_columns)
Test_df = SplitDatetime(Test_df,Time_columns)

In [19]:
Train_df.head(1)

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Pickup - Day of Month,Pickup - Weekday (Mo = 1),Distance (KM),Temperature,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Placement - Time_hour,Placement - Time_minutes,Placement - Time_seconds,Confirmation - Time_hour,Confirmation - Time_minutes,Confirmation - Time_seconds,Arrival at Pickup - Time_hour,Arrival at Pickup - Time_minutes,Arrival at Pickup - Time_seconds,Pickup - Time_hour,Pickup - Time_minutes,Pickup - Time_seconds
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,9,5,9,5,9,5,4,20.4,-1.317755,36.83037,-1.300406,36.829741,Rider_Id_432,745,1637,1309,13.8,549,9,35,46,9,40,10,10,4,47,10,27,30


**Taking Care of missing Data

In [20]:
from sklearn.impute import SimpleImputer

def missingData(df,columns):
  imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
  imputer.fit(df[columns])
  df[columns] = imputer.transform(df[columns])
  return df

In [21]:
columns = ['Temperature']
Test_df = missingData(Test_df,columns)
Train_df = missingData(Train_df,columns)

In [22]:
Train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21201 entries, 0 to 21200
Data columns (total 37 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Order No                              21201 non-null  object 
 1   User Id                               21201 non-null  object 
 2   Vehicle Type                          21201 non-null  object 
 3   Platform Type                         21201 non-null  object 
 4   Personal or Business                  21201 non-null  object 
 5   Placement - Day of Month              21201 non-null  int64  
 6   Placement - Weekday (Mo = 1)          21201 non-null  int64  
 7   Confirmation - Day of Month           21201 non-null  int64  
 8   Confirmation - Weekday (Mo = 1)       21201 non-null  int64  
 9   Arrival at Pickup - Day of Month      21201 non-null  int64  
 10  Arrival at Pickup - Weekday (Mo = 1)  21201 non-null  int64  
 11  Pickup - Day of

# **Feature Engineering**

**Get Dummy Variables**

In [23]:
def dummy_encoder(df, cols):
  dummies = pd.get_dummies(df,columns= cols,drop_first = True)
  merged = pd.concat([df,dummies],axis='columns')
  final = merged.drop(cols, axis = 1)
  return dummies

In [24]:
cols = ['Platform Type','Personal or Business', 'Placement - Weekday (Mo = 1)',
        'Confirmation - Weekday (Mo = 1)','Pickup - Weekday (Mo = 1)','Rider Id','Vehicle Type','User Id']
Train_df = dummy_encoder(Train_df, cols)
Test_df = dummy_encoder(Test_df, cols)
Test_df.head(1)

,Order No,Placement - Day of Month,Confirmation - Day of Month,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Pickup - Day of Month,Distance (KM),Temperature,Pickup Lat,Pickup Long,Destination Lat,Destination Long,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Placement - Time_hour,Placement - Time_minutes,Placement - Time_seconds,Confirmation - Time_hour,Confirmation - Time_minutes,Confirmation - Time_seconds,Arrival at Pickup - Time_hour,Arrival at Pickup - Time_minutes,Arrival at Pickup - Time_seconds,Pickup - Time_hour,Pickup - Time_minutes,Pickup - Time_seconds,Platform Type_2,Platform Type_3,Platform Type_4,Personal or Business_Personal,Placement - Weekday (Mo = 1)_2,Placement - Weekday (Mo = 1)_3,Placement - Weekday (Mo = 1)_4,Placement - Weekday (Mo = 1)_5,Placement - Weekday (Mo = 1)_6,Placement - Weekday (Mo = 1)_7,Confirmation - Weekday (Mo = 1)_2,Confirmation - Weekday (Mo = 1)_3,...,User Id_User_Id_909,User Id_User_Id_910,User Id_User_Id_912,User Id_User_Id_913,User Id_User_Id_914,User Id_User_Id_915,User Id_User_Id_917,User Id_User_Id_918,User Id_User_Id_919,User Id_User_Id_920,User Id_User_Id_922,User Id_User_Id_924,User Id_User_Id_925,User Id_User_Id_928,User Id_User_Id_93,User Id_User_Id_930,User Id_User_Id_935,User Id_User_Id_937,User Id_User_Id_938,User Id_User_Id_94,User Id_User_Id_941,User Id_User_Id_944,User Id_User_Id_948,User Id_User_Id_95,User Id_User_Id_953,User Id_User_Id_954,User Id_User_Id_955,User Id_User_Id_957,User Id_User_Id_969,User Id_User_Id_97,User Id_User_Id_970,User Id_User_Id_972,User Id_User_Id_973,User Id_User_Id_978,User Id_User_Id_980,User Id_User_Id_981,User Id_User_Id_991,User Id_User_Id_992,User Id_User_Id_995,User Id_User_Id_996
0,Order_No_19248,27,27,27,3,27,8,23.24612,-1.333275,36.870815,-1.305249,36.82239,439,1511,13.3,171,16,44,10,16,44,29,16,53,4,17,6,47,0,1,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
Train_df.head(1)

,Order No,Placement - Day of Month,Confirmation - Day of Month,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Pickup - Day of Month,Distance (KM),Temperature,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Time from Pickup to Arrival,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Placement - Time_hour,Placement - Time_minutes,Placement - Time_seconds,Confirmation - Time_hour,Confirmation - Time_minutes,Confirmation - Time_seconds,Arrival at Pickup - Time_hour,Arrival at Pickup - Time_minutes,Arrival at Pickup - Time_seconds,Pickup - Time_hour,Pickup - Time_minutes,Pickup - Time_seconds,Platform Type_2,Platform Type_3,Platform Type_4,Personal or Business_Personal,Placement - Weekday (Mo = 1)_2,Placement - Weekday (Mo = 1)_3,Placement - Weekday (Mo = 1)_4,Placement - Weekday (Mo = 1)_5,Placement - Weekday (Mo = 1)_6,Placement - Weekday (Mo = 1)_7,Confirmation - Weekday (Mo = 1)_2,...,User Id_User_Id_96,User Id_User_Id_960,User Id_User_Id_961,User Id_User_Id_962,User Id_User_Id_963,User Id_User_Id_964,User Id_User_Id_965,User Id_User_Id_966,User Id_User_Id_967,User Id_User_Id_968,User Id_User_Id_969,User Id_User_Id_97,User Id_User_Id_970,User Id_User_Id_971,User Id_User_Id_972,User Id_User_Id_974,User Id_User_Id_975,User Id_User_Id_976,User Id_User_Id_977,User Id_User_Id_978,User Id_User_Id_979,User Id_User_Id_98,User Id_User_Id_981,User Id_User_Id_982,User Id_User_Id_983,User Id_User_Id_984,User Id_User_Id_985,User Id_User_Id_986,User Id_User_Id_987,User Id_User_Id_988,User Id_User_Id_989,User Id_User_Id_99,User Id_User_Id_990,User Id_User_Id_991,User Id_User_Id_993,User Id_User_Id_994,User Id_User_Id_995,User Id_User_Id_997,User Id_User_Id_998,User Id_User_Id_999
0,Order_No_4211,9,9,9,5,9,4,20.4,-1.317755,36.83037,-1.300406,36.829741,745,1637,1309,13.8,549,9,35,46,9,40,10,10,4,47,10,27,30,0,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Variable Selection by Correlation and Significance**

In [26]:
Train = Train_df.drop(['Order No'],axis=1)

In [ ]:
# Calculate correlations between predictor variables and the response variable
corrs = Train.corr()['Time from Pickup to Arrival'].sort_values(ascending=False)

In [ ]:
from scipy.stats import pearsonr

# Build a dictionary of correlation coefficients and p-values
dict_cp = {}

column_titles = [col for col in corrs.index if col!= 'Time from Pickup to Arrival']
for col in column_titles:
    p_val = round(pearsonr(Train[col], Train['Time from Pickup to Arrival'])[1],6)
    dict_cp[col] = {'Correlation_Coefficient':corrs[col],
                    'P_Value':p_val}

df_cp = pd.DataFrame(dict_cp).T
df_cp_sorted = df_cp.sort_values('P_Value')
df_cp_sorted[df_cp_sorted['P_Value']<0.1]

In [ ]:
# The dependent variable remains the same:
y_data = Train['Time from Pickup to Arrival'] 

# Model building - Independent Variable (IV) DataFrame
X_names = list(df_cp[df_cp['P_Value'] < 0.05].index)
X_data = Train[X_names]

In [ ]:
X_data.head()

In [ ]:
# Create the correlation matrix
corr = X_data.corr()

# Find rows and columns where correlation coefficients > 0.9 or <-0.9
corr[np.abs(corr) > 0.9]

In [ ]:
# As before, we create the correlation matrix
# and find rows and columnd where correlation coefficients > 0.9 or <-0.9
corr = X_data.corr()
r, c = np.where(np.abs(corr) > 0.9)

# We are only interested in the off diagonal entries:
off_diagonal = np.where(r != c)

# Show the correlation matrix rows and columns where we have highly correlated off diagonal entries:
corr.iloc[r[off_diagonal], c[off_diagonal]]

In [ ]:
DropColumns = ['Pickup - Weekday (Mo = 1)_5', 'Placement - Weekday (Mo = 1)_5','Placement - Weekday (Mo = 1)_5',
               'Confirmation - Weekday (Mo = 1)_5', 'Confirmation - Weekday (Mo = 1)_5', 'Confirmation - Weekday (Mo = 1)_2',
               'Pickup - Weekday (Mo = 1)_2','Pickup - Weekday (Mo = 1)_2', 'Placement - Weekday (Mo = 1)_2',
               'Placement - Weekday (Mo = 1)_2','Rider Id_Rider_Id_83','Confirmation - Day of Month','Confirmation - Day of Month',
               'Confirmation - Day of Month','Arrival at Pickup - Day of Month','Arrival at Pickup - Day of Month',
               'Arrival at Pickup - Day of Month','Pickup - Day of Month','Pickup - Day of Month',
               'Placement - Day of Month','Placement - Day of Month','Placement - Weekday (Mo = 1)_6',
               'Pickup - Weekday (Mo = 1)_6','Pickup - Weekday (Mo = 1)_6','Confirmation - Weekday (Mo = 1)_6',
               'Confirmation - Weekday (Mo = 1)_6']

In [ ]:
X_data = X_data.drop(DropColumns,axis=1)

# Ensemble Method


**Feature Scaling**

In [ ]:
# import the scaling module
from sklearn.preprocessing import StandardScaler

In [ ]:
# create standardization object
scaler_X = StandardScaler()
scaler_y = StandardScaler()

In [ ]:
columns = ['Distance (KM)', 'Destination Long','Pickup Long']
# save standardized features into new variable
X_data[columns] = scaler_X.fit_transform(X_data[columns])
y_data = np.array(y_data).reshape(-1,1)


**splitting features into test set and training set**

In [ ]:
# import train/test split module
from sklearn.model_selection import train_test_split

In [ ]:
# split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_data,
                                                    y_data,
                                                    test_size=0.20,
                                                    random_state=1)

**Ridge Model**

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge

In [ ]:
ridge = Ridge()
regressor = LinearRegression()

In [ ]:
ridge.fit(X_train, y_train)
regressor.fit(X_train, y_train)

In [ ]:
ridge_pred = ridge.predict(X_test)
regressor_pred = regressor.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
ridge_rmse = mean_squared_error(y_test, ridge_pred, squared= False)
regressor_rmse = mean_squared_error(y_test, regressor_pred, squared= False)

In [ ]:
ridge_rmse

In [ ]:
regressor_rmse

Building Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor
reg_tree = DecisionTreeRegressor(max_depth=3)

In [ ]:
reg_tree.fit(X_train,y_train)
dt_pred = reg_tree.predict(X_test)

In [ ]:
dt_rmse = mean_squared_error(y_test, dt_pred, squared= False)

In [ ]:
dt_rmse

SVR Model

In [ ]:
from sklearn.svm import SVR
sv_reg = SVR(kernel='rbf', gamma='auto')
sv_reg.fit(X_train,y_train)
svr_pred = sv_reg.predict(X_test)

In [ ]:
svr_rmse = mean_squared_error(y_test, svr_pred, squared= False)

In [ ]:
svr_rmse

Lasso Model

In [ ]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.01)
lasso.fit(X_train, y_train)
y_pred = lasso.predict(X_test)

In [ ]:
lasso_rmse = mean_squared_error(y_test, y_pred, squared= False)
lasso_rmse

Stacking method

In [ ]:
from sklearn.ensemble import StackingRegressor
models = [("LR",regressor),("DT",reg_tree),("SVR",sv_reg),("Ridge",ridge),("lasso",lasso)]

In [ ]:
meta_learner_reg = LinearRegression()
s_reg = StackingRegressor(estimators=models, final_estimator=meta_learner_reg)

In [ ]:
s_reg.fit(X_train,y_train)
y_pred = s_reg.predict(X_test)

In [ ]:
meta_rmse = mean_squared_error(y_test, y_pred, squared= False)
meta_rmse